In [1]:
import h2o

In [2]:
from h2o.automl import H2OAutoML, get_leaderboard

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,30 mins 38 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,1 month and 4 days
H2O cluster name:,H2O_from_python_deepa_b5kbqu
H2O cluster total nodes:,1
H2O cluster free memory:,1.240 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [4]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
train.head()

response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
1,0.869293,-0.635082,0.22569,0.32747,-0.689993,0.754202,-0.248573,-1.09206,0,1.37499,-0.653674,0.930349,1.10744,1.1389,-1.5782,-1.04699,0,0.65793,-0.0104546,-0.0457672,3.10196,1.35376,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,0.907542,0.329147,0.359412,1.49797,-0.31301,1.09553,-0.557525,-1.58823,2.17308,0.812581,-0.213642,1.27101,2.21487,0.499994,-1.26143,0.732156,0,0.398701,-1.13893,-0.00081911,0,0.30222,0.833048,0.9857,0.978098,0.779732,0.992356,0.798343
1,0.798835,1.47064,-1.63597,0.453773,0.425629,1.10487,1.28232,1.38166,0,0.851737,1.54066,-0.81969,2.21487,0.99349,0.35608,-0.208778,2.54822,1.25695,1.12885,0.900461,0,0.909753,1.10833,0.985692,0.951331,0.803252,0.865924,0.780118
0,1.34438,-0.876626,0.935913,1.99205,0.882454,1.78607,-1.64678,-0.942383,0,2.42326,-0.676016,0.736159,2.21487,1.29872,-1.43074,-0.364658,0,0.745313,-0.678379,-1.36036,0,0.946652,1.0287,0.998656,0.728281,0.8692,1.02674,0.957904
1,1.10501,0.321356,1.5224,0.882808,-1.20535,0.681466,-1.07046,-0.921871,0,0.800872,1.02097,0.971407,2.21487,0.596761,-0.350273,0.631194,0,0.479999,-0.373566,0.113041,0,0.755856,1.36106,0.98661,0.838085,1.1333,0.872245,0.808487
0,1.59584,-0.607811,0.00707492,1.81845,-0.111906,0.84755,-0.566437,1.58124,2.17308,0.755421,0.64311,1.42637,0,0.921661,-1.19043,-1.61559,0,0.651114,-0.654227,-1.27434,3.10196,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818
1,0.409391,-1.88468,-1.02729,1.67245,-1.6046,1.33801,0.0554274,0.0134659,2.17308,0.509783,-1.03834,0.707862,0,0.746918,-0.358465,-1.64665,0,0.367058,0.0694965,1.37713,3.10196,0.869418,1.22208,1.00063,0.545045,0.698653,0.977314,0.828786
1,0.933895,0.62913,0.527535,0.238033,-0.966569,0.547811,-0.0594392,-1.70687,2.17308,0.941003,-2.65373,-0.15722,0,1.03037,-0.175505,0.523021,2.54822,1.37355,1.29125,-1.46745,0,0.901837,1.08367,0.979696,0.7833,0.849195,0.894356,0.774879
1,1.40514,0.536603,0.689554,1.17957,-0.110061,3.2024,-1.52696,-1.57603,0,2.93154,0.567342,-0.130033,2.21487,1.78712,0.899499,0.585151,2.54822,0.401865,-0.151202,1.16349,0,1.66707,4.03927,1.17583,1.04535,1.54297,3.53483,2.74075
1,1.17657,0.104161,1.397,0.479721,0.265513,1.13556,1.53483,-0.253291,0,1.02725,0.534316,1.18002,0,2.40566,0.0875568,-0.976534,2.54822,1.25038,0.268541,0.530334,0,0.833175,0.773968,0.98575,1.1037,0.84914,0.937104,0.812364


In [7]:
test.head()

response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
0,0.623878,0.459659,-0.316512,1.55701,0.644784,0.455929,-2.24983,-0.361949,0,0.832978,1.41147,1.52846,0,0.722857,1.00964,0.999539,0,1.19099,0.81404,-0.879803,3.10196,0.583344,0.790246,1.04087,0.848858,0.229702,0.65594,0.677857
0,1.68204,0.774251,1.32875,0.234729,-0.55783,0.442921,-0.0832048,1.08119,2.17308,1.14585,0.16228,-0.268187,0,1.21407,0.194966,-1.15627,2.54822,1.01915,-0.759163,0.136347,0,0.94183,0.941976,0.98816,0.86435,0.835132,0.767871,0.833013
1,1.09403,-0.88734,0.949229,0.410261,-1.55868,0.523352,0.131675,0.304513,0,1.11916,1.09383,-0.396352,2.21487,0.6289,0.628245,1.45775,0,0.523082,0.508394,0.988137,0,0.929989,0.973742,0.989141,0.780519,0.749821,0.887545,0.925278
0,1.33871,-0.899027,-1.25534,0.884285,-0.747925,0.547445,-0.79518,0.174235,2.17308,0.771663,-0.31855,1.02356,2.21487,0.408495,-1.1431,0.139699,0,1.29395,-1.34131,1.61685,0,0.79204,0.76921,0.984082,1.06928,0.700852,0.83083,0.71671
0,0.699095,1.43363,-1.72253,0.650113,-0.167574,0.967464,0.737696,-0.886391,2.17308,1.66797,0.446892,0.851008,0,0.59641,0.156736,0.340513,0,0.694684,-0.00046069,-1.14949,3.10196,0.714182,0.839843,0.984067,0.917766,0.388553,0.91664,0.886331
1,0.842208,-0.144202,-1.702,0.685875,0.544338,0.578774,-0.221837,-0.586474,1.08654,0.909401,2.81316,-0.0706666,0,1.54986,-0.85182,1.58811,0,1.14986,0.590011,0.0597691,0,0.872279,0.684341,0.99032,0.510151,0.720446,0.555454,0.535795
1,0.456242,1.74043,0.258427,1.45223,-0.0513728,0.675328,-0.797161,-1.34763,2.17308,0.56858,-0.482712,-1.73738,0,0.539333,1.17621,1.06666,0,0.980687,-0.125384,0.0869597,3.10196,0.994118,0.960552,1.00353,0.57609,0.876509,0.85138,0.801164
0,0.933712,1.35084,0.773338,0.532638,-0.679537,0.718017,0.745618,-1.4962,1.08654,0.274218,1.07537,1.56341,0,0.39146,-0.59422,0.170209,2.54822,0.573711,2.12157,0.256763,0,0.625972,0.773067,0.987348,0.674946,0.829309,0.645423,0.597715
0,0.7335,-0.998372,-1.68702,0.951169,-0.591779,1.14912,-0.942724,-0.424039,2.17308,1.04185,1.10063,1.27434,0,0.713549,0.0201984,0.343287,2.54822,1.44706,0.165271,1.40599,0,0.798081,0.852638,0.990337,0.799596,0.918739,1.27283,1.07795
0,0.968118,1.719,0.030934,1.80122,-0.867323,0.631357,-0.810034,1.20648,0,0.88699,0.00103165,1.57395,0,1.02721,0.512643,0.381009,2.54822,1.08511,0.54837,-0.86704,3.10196,0.844315,1.03113,1.32327,1.11639,0.728902,0.805447,1.19234


In [8]:
# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

In [9]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [10]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
00:39:02.388: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████Failed polling AutoML progress log: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\deepa\\AppData\\Local\\Temp\\tmps1mqrinl.csv'
█Failed polling AutoML progress log: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\deepa\\AppData\\Local\\Temp\\tmp3twdktkp.csv'
█| 100%


In [11]:
# AutoML Leaderboard
lb = aml.leaderboard

In [12]:
# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

In [13]:
# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20200121_003902,0.787947,0.553556,0.804322,0.328047,0.433203,0.187665,1049,0.07299
StackedEnsemble_BestOfFamily_AutoML_20200121_003902,0.785796,0.555109,0.803425,0.329251,0.434021,0.188374,415,0.018209
GBM_grid__1_AutoML_20200121_003902_model_2,0.781247,0.578962,0.801218,0.334139,0.443313,0.196526,54291,0.005725
GBM_5_AutoML_20200121_003902,0.780862,0.559708,0.79783,0.325399,0.436083,0.190168,5440,0.002429
GBM_1_AutoML_20200121_003902,0.778997,0.56159,0.796523,0.326697,0.437003,0.190971,821,0.002401
GBM_2_AutoML_20200121_003902,0.778338,0.561527,0.79632,0.329805,0.437199,0.191143,822,0.002291
GBM_3_AutoML_20200121_003902,0.776389,0.563906,0.793284,0.328065,0.438274,0.192084,509,0.002502
GBM_grid__1_AutoML_20200121_003902_model_1,0.775408,0.586612,0.794803,0.329128,0.447083,0.199883,78020,0.00693
GBM_4_AutoML_20200121_003902,0.770758,0.570912,0.790371,0.353743,0.441681,0.195082,5115,0.002732
GBM_grid__1_AutoML_20200121_003902_model_4,0.770053,0.570409,0.788463,0.362283,0.441121,0.194587,1139,0.003006


In [14]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200121_003902

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.08106582734231055
RMSE: 0.28472061278086375
LogLoss: 0.30624278815775713
Null degrees of freedom: 9999
Residual degrees of freedom: 9983
Null deviance: 13828.11338742432
Residual deviance: 6124.855763155143
AIC: 6158.855763155143
AUC: 0.9835935892842987
AUCPR: 0.9842006301293156
Gini: 0.9671871785685975

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5057700896352938: 


,0,1,Error,Rate
0,4295.0,410.0,0.0871,(410.0/4705.0)
1,275.0,5020.0,0.0519,(275.0/5295.0)
Total,4570.0,5430.0,0.0685,(685.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


metric,threshold,value,idx
max f1,0.5057701,0.9361305,202.0
max f2,0.3990011,0.9604655,241.0
max f0point5,0.6140574,0.9428832,160.0
max accuracy,0.5078956,0.9315,201.0
max precision,0.9461384,1.0,0.0
max recall,0.2392266,1.0,306.0
max specificity,0.9461384,1.0,0.0
max absolute_mcc,0.5057701,0.8626190,202.0
max min_per_class_accuracy,0.5310427,0.9300744,191.0
max mean_per_class_accuracy,0.5250584,0.9308060,194.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.66 %


...

,0,1,Error,Rate
0,2145.0,2560.0,0.5441,(2560.0/4705.0)
1,593.0,4702.0,0.112,(593.0/5295.0)
Total,2738.0,7262.0,0.3153,(3153.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


metric,threshold,value,idx
max f1,0.3289935,0.7489050,282.0
max f2,0.1635485,0.8610287,355.0
max f0point5,0.5891570,0.7366771,167.0
max accuracy,0.5498542,0.712,187.0
max precision,0.9409014,1.0,0.0
max recall,0.0717869,1.0,393.0
max specificity,0.9409014,1.0,0.0
max absolute_mcc,0.5498542,0.4257269,187.0
max min_per_class_accuracy,0.5295946,0.7106704,196.0
max mean_per_class_accuracy,0.5498542,0.7132083,187.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.94 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.01,0.9180085,1.8508026,1.8508026,0.98,0.9261170,0.98,0.9261170,0.0185080,0.0185080,85.0802644,85.0802644
,2,0.02,0.9084254,1.8130312,1.8319169,0.96,0.9127385,0.97,0.9194278,0.0181303,0.0366383,81.3031161,83.1916903
,3,0.03,0.9014909,1.7374882,1.8004407,0.92,0.9049494,0.9533333,0.9146016,0.0173749,0.0540132,73.7488196,80.0440667
,4,0.04,0.8946282,1.8130312,1.8035883,0.96,0.8977575,0.955,0.9103906,0.0181303,0.0721435,81.3031161,80.3588291
,5,0.05,0.8892088,1.7374882,1.7903683,0.92,0.8919210,0.948,0.9066967,0.0173749,0.0895184,73.7488196,79.0368272
,6,0.1,0.8612864,1.7034939,1.7469311,0.902,0.8757719,0.925,0.8912343,0.0851747,0.1746931,70.3493862,74.6931067
,7,0.15,0.8298720,1.6015109,1.6984577,0.848,0.8461960,0.8993333,0.8762216,0.0800755,0.2547686,60.1510859,69.8457664
,8,0.2,0.7957939,1.5372993,1.6581681,0.814,0.8133868,0.878,0.8605129,0.0768650,0.3316336,53.7299339,65.8168083
,9,0.3,0.7219335,1.4032106,1.5731822,0.743,0.7609567,0.833,0.8273275,0.1403211,0.4719547,40.3210576,57.3182247
,10,0.4,0.6361097,1.1992446,1.4796978,0.635,0.6794052,0.7835,0.7903469,0.1199245,0.5918791,19.9244570,47.9697828


In [15]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [19]:
preds.head(10)

predict,p0,p1
0,0.73885,0.26115
0,0.694848,0.305152
1,0.562527,0.437473
1,0.260655,0.739345
0,0.653915,0.346085
1,0.249595,0.750405
1,0.257673,0.742327
0,0.638967,0.361033
1,0.585796,0.414204
0,0.808902,0.191098
